## Se planean crear 2 tipos de modelos, comparar y luego elegir el mejor

## 1. Random Forest

### imports

In [28]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [29]:
# read the data
df_mean = pd.read_csv('assets/cleaned_train.csv')

In [30]:
#df_mean.info()
df_mean.head()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,29.833333,10.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.254000
1,2,29.083333,2.0,16.625,13.666667,17.375,21.333333,22.666667,20.375000,22.916667,...,1.000000,0.380208,0.119792,0.416667,0.781250,-0.288187,-1.448893,-0.319096,1.116661,1.016001
2,3,30.750000,10.0,0.000,0.000000,0.000,8.500000,0.000000,0.000000,0.000000,...,0.801667,0.000000,0.000000,0.000000,2.062500,0.000000,0.000000,0.000000,0.000000,26.162014
3,4,28.153846,9.0,26.600,20.071429,25.800,30.269231,26.666667,21.090909,25.115385,...,1.015833,-1.125000,0.000000,0.500000,1.515625,7.029999,0.000000,0.000000,6.329994,4.064002
4,5,28.714286,13.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,774.700440


In [31]:
# Set data frames to be the inputs of the skit model
X = df_mean.iloc[:, :-1]
Y = df_mean.iloc[:, 23:]
Y = Y.values.ravel()

In [32]:
#   Scaling data frame
# scaling X
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [33]:
# splits data

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)


In [34]:
# Fit the model

clf = RandomForestRegressor(random_state=42, max_depth=35, n_jobs=-1)
clf.fit(X_train, Y_train)

RandomForestRegressor(max_depth=35, n_jobs=-1, random_state=42)

In [35]:
# Get predictions and evaluate performance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Predecir valores usando el modelo entrenado
y_pred = clf.predict(X_test)

# Calcular métricas de regresión
mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Imprimir resultados
# MSE: Un MSE bajo indica que el modelo predice los valores reales con mayor precisión.
print("Mean Squared Error (MSE):", mse)
# MAE:  Un MAE bajo indica que el error promedio entre los valores predichos y reales es pequeño.
# En general, se busca un MAE lo más cercano a 0 posible. 
print("Mean Absolute Error (MAE):", mae)
# R-squared: Un valor de R-cuadrado cercano a 1 
# indica que el modelo explica una gran proporción de la variabilidad en los datos.
print("R-squared:", r2)

rfr_m = [mse, mae, r2]

Mean Squared Error (MSE): 14802.15123722491
Mean Absolute Error (MAE): 32.486672054971365
R-squared: 0.5258692789469954


## Neural Network


In [2]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [3]:
# read the data
df_mean = pd.read_csv('assets/cleaned_train.csv')

In [4]:
# Set data frames to be the inputs of the skit model
X = df_mean.iloc[:, :-1]
Y = df_mean.iloc[:, 23:]
Y.describe()

,Expected
count,1.143831e+06
mean,3.681348e+01
std,1.774018e+02
min,1.000000e-02
25%,2.540001e-01
50%,1.016001e+00
75%,3.302002e+00
max,1.939799e+03


In [5]:
#   Scaling data frame
# scaling X
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [6]:
# splits data

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [7]:
import numpy as np
# Ensure Y_train is a 2D array
Y_train = np.reshape(Y_train, (-1, 1))

In [8]:
print(X_train.shape, len(X_train), type(X_train))
print(Y_train.shape, len(Y_train), type(Y_train))

(1029447, 23) 1029447 <class 'numpy.ndarray'>
(1029447, 1) 1029447 <class 'numpy.ndarray'>


In [9]:
#!pip install keras-tuner


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError


input_shape = X_train.shape[1]

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from kerastuner.tuners import BayesianOptimization

# ... (your data and other imports)

def build_model(hp):
  model = Sequential()
  model.add(Dense(hp.Int('units_1', min_value=32, max_value=128, step=32), input_shape=(input_shape,), activation='relu'))
  model.add(Dense(hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu'))
  model.add(Dense(1, activation='linear'))

  model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss='mean_squared_error',
                metrics=[MeanSquaredError()])
  return model

tuner = BayesianOptimization(
    build_model,
    objective='val_mean_squared_error',
    max_trials=10,  # Adjust the number of trials as needed
    executions_per_trial=2,  # Optional: Run each trial multiple times
    directory='my_dir',  # Optional: Specify a directory to save results
    project_name='my_project'
)

tuner.search(x=X_train, y=Y_train, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

# Build the best model
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, Y_train, epochs=100, validation_split=0.2)



/home/kaforerof/.pyenv/versions/3.11.9/envs/introduction/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |units_1
128               |128               |units_2
0.01              |0.01              |learning_rate

Epoch 1/10
25737/25737 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - loss: 29445.3945 - mean_squared_error: 29445.3945 - val_loss: 28806.2539 - val_mean_squared_error: 28806.2539
Epoch 2/10
25737/25737 ━━━━━━━━━━━━━━━━━━━━ 59s 2ms/step - loss: 28178.9434 - mean_squared_error: 28178.9434 - val_loss: 28349.2910 - val_mean_squared_error: 28349.2910
Epoch 3/10
25728/25737 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27744.7305 - mean_squared_error: 27744.7305

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, R2Score

# Evaluate the model
y_pred = ann.predict(X_test)

# Calculate metrics
mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print results
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared:", r2)
ann_m = [mse, mae, r2]

3575/3575 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step
Mean Squared Error (MSE): 32505.781321358692
Mean Absolute Error (MAE): 36.49329975911739
R-squared: -0.041199326515197754


##compare models

In [ ]:
print(f'[mse, mae, r2]')
#print(f'Random forest: {rfr_m}')
print(f'NN (Keras): {ann_m}')

[mse, mae, r2]


NameError: name 'rfr_m' is not defined